In [1]:
from langchain_openai import ChatOpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools import PythonREPLTool
from langchain.hub import pull as hub_pull
from dotenv import load_dotenv
import os
import logging
import pandas as pd
import json

In [2]:
# Load environment variables
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check for API key
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please check your .env file.")

# Logging configuration
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [3]:
# Initialize LLM with settings
def initialize_llm():
    return ChatOpenAI(
        temperature=0.1,
        model="gpt-4o",
        api_key=api_key,
        verbose=True
    )

# Create ReAct agent with tool cache, adding Python execution for DataFrame support
def create_react_agent(llm):
    try:
        react_prompt_template = hub_pull("hwchase17/react")
        tools = [PythonREPLTool()]
        agent = initialize_agent(
            tools,
            llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True,
            agent_kwargs={"prompt": react_prompt_template}
        )
        return agent
    except Exception as e:
        logger.error("Error creating ReAct agent: %s", e)
        raise

# Function to run analysis and output DataFrame structure
def analyze_with_dataframe(question, agent):
    try:
        response = agent.run(question)
        # Parse the JSON response into a DataFrame
        data = json.loads(response)
        df = pd.DataFrame(data)
        return df
    except json.JSONDecodeError as e:
        logger.error("Error parsing response to JSON: %s", e)
        return f"Error parsing response: {str(e)}"
    except Exception as e:
        logger.error("Error during analysis with DataFrame: %s", e)
        return f"Error during analysis: {str(e)}", None

In [4]:
# INPUTS
ramo_empresa = 'Construção'
direcionadores = 'Redução de Custo'
nome_processo = 'Suprir materiais'
atividade = 'Realizar compra'
evento = 'Valor gasto é maior que foi orçado para a área'
causa = 'Ordem de compra não é vinculada ao orçamento'

# Carregamento e formatação das métricas da empresa
company_metrics = f"""**Ramo da Empresa**: {ramo_empresa}
**Direcionadores**: {direcionadores}
**Nome do Processo**: {nome_processo}
**Atividade**: {atividade}
**Evento**: {evento}
**Causa**: {causa}
"""

In [ ]:
# Initialize LLM and ReAct agent
llm = initialize_llm()
agent = create_react_agent(llm)

# Reestruturação da pergunta para gerar resposta em formato JSON
question =  f"""
Você é um consultor sênior altamente qualificado, especializado em análise de desempenho organizacional para grandes empresas, com foco em **{ramo_empresa}**. Sua tarefa é analisar dados da empresa e fornecer recomendações objetivas de melhoria usando as melhores práticas de consultoria.

**OBJETIVO**:
Usando as métricas do DataFrame **{company_metrics}**, identifique problemas críticos e oportunidades de melhoria na empresa.

**INSTRUÇÕES**:
1. Avalie o impacto de **{causa}** nas operações e considere ferramentas usadas, como Ordem de Serviço, DRE, Ordem de Compras e CRM.
2. Priorize recomendações com alto **ROI** e detalhe a **quantificação de benefícios**, com métricas estimadas de impacto, como percentuais ou valores financeiros.
3. Inclua **possíveis riscos de implementação** para cada solução e estratégias breves de mitigação para aumentar as chances de sucesso.

**METODOLOGIA**:
1. Use metodologias como **Six Sigma, Lean** e **DMAIC** para estruturar a análise de problemas e melhorias.
2. Fundamente suas recomendações com **casos de sucesso do setor**, quando relevante, e proponha melhorias escalonadas para as recomendações que exigem maior investimento inicial.
3. Priorize soluções que sejam **práticas e de implementação viável** dentro das operações atuais da empresa.

**PASSOS PARA ANÁLISE**:
1. Revise a situação atual usando os indicadores do DataFrame.
2. Identifique, com base nos **{direcionadores}**, ao menos **5** problemas que impactam o desempenho em relação a **{evento}**.
3. Para cada problema, proponha uma oportunidade de melhoria incluindo os seguintes elementos:

   - **Problema Mapeado**: Descrição objetiva do problema identificado.
   - **Oportunidade de Melhoria**: Sugestão clara da oportunidade de melhoria para o problema.
   - **Solução**: Descrição detalhada da solução, incluindo especificação de etapas escalonadas, se relevante.
   - **Backlog de Atividades**: Lista de tarefas específicas que devem ser adicionadas ao backlog de implementação.
   - **Investimento**: Estimativa de horas e recursos para desenvolvimento e implantação.
   - **Ganhos Quantificados**: Benefícios esperados em termos percentuais ou valores financeiros.
   - **Riscos e Mitigação**: Descrição breve dos riscos de implementação e estratégia para mitigá-los.

**FORMATO DE RESPOSTA**:
- Utilize **PythonREPLTool** para gerar uma lista de dicionários Python, conforme o exemplo:
[
    {{
        "problema_mapeado": "Descrição do problema",
        "oportunidade_melhoria": "Descrição da oportunidade de melhoria",
        "solucao": "Descrição da solução proposta",
        "backlog_atividades": ["Atividade 1", "Atividade 2", "..."],
        "investimento": "Estimativa do investimento necessário em horas",
        "ganhos_quantificados": "Benefícios esperados em termos percentuais",
        "riscos_e_mitigacao": "Breve descrição dos riscos e das estratégias de mitigação"
    }},
    // outras oportunidades de melhoria
]
- Retorne apenas a lista de dicionários sem aspas adicionais, sem a palavra "'''python" ou "'''json", sem variável e sem explicações.
- Caso seja gerado como string ou variável, transforme diretamente em uma lista de dicionários.
"""

# Execute a análise e gere o DataFrame
analysis_df = analyze_with_dataframe(question, agent)

c:\Users\PedroLustosa\AppData\Local\Programs\Python\Python311\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
C:\Users\PedroLustosa\AppData\Local\Temp\ipykernel_18604\2950299714.py:15: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(
C:\Users\PedroLustosa\AppData\Local\Temp\ipykernel_18604\2950299714.py:31: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(question)




> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Para abordar a análise de desempenho organizacional e fornecer recomendações objetivas de melhoria, seguirei as instruções fornecidas e utilizarei as metodologias sugeridas. Vou estruturar a resposta em uma lista de dicionários Python, conforme o formato solicitado.

Action: Python_REPL
Action Input:
```python
[
    {
        "problema_mapeado": "Ordem de compra não é vinculada ao orçamento, resultando em gastos excessivos.",
        "oportunidade_melhoria": "Implementar um sistema de vinculação automática entre ordens de compra e orçamento.",
        "solucao": "Desenvolver e integrar um módulo no sistema ERP que vincule automaticamente cada ordem de compra ao orçamento correspondente, com alertas para desvios.",
        "backlog_atividades": [
            "Analisar requisitos do sistema atual",
            "Desenvolver módulo de vinculação",
            "Testar integração com ERP",
            "Treinar equipe para uso do novo módulo"
        ],
        "investimento": "200 horas de d

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[
    {
        "problema_mapeado": "Ordem de compra não é vinculada ao orçamento, resultando em gastos excessivos.",
        "oportunidade_melhoria": "Implementar um sistema de vinculação automática entre ordens de compra e orçamento.",
        "solucao": "Desenvolver e integrar um módulo no sistema ERP que vincule automaticamente cada ordem de compra ao orçamento correspondente, com alertas para desvios.",
        "backlog_atividades": [
            "Analisar requisitos do sistema atual",
            "Desenvolver módulo de vinculação",
            "Testar integração com ERP",
            "Treinar equipe para uso do novo módulo"
        ],
        "investimento": "200 horas de desenvolvimento e integração",
        "ganhos_quantificados": "Redução de 15% nos gastos excessivos",
        "riscos_e_mitigacao": "Risco de resistência da equipe; mitigar com treinamento e comunicação clara dos benefícios."
    },
    {
        "problema_mapeado": "Falta de visibilidade em tempo real dos gast

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


I will provide the final answer directly as a list of dictionaries, following the specified format.

Final Answer:
[
    {
        "problema_mapeado": "Ordem de compra não é vinculada ao orçamento, resultando em gastos excessivos.",
        "oportunidade_melhoria": "Implementar um sistema de vinculação automática entre ordens de compra e orçamento.",
        "solucao": "Desenvolver e integrar um módulo no sistema ERP que vincule automaticamente cada ordem de compra ao orçamento correspondente, com alertas para desvios.",
        "backlog_atividades": [
            "Analisar requisitos do sistema atual",
            "Desenvolver módulo de vinculação",
            "Testar integração com ERP",
            "Treinar equipe para uso do novo módulo"
        ],
        "investimento": "200 horas de desenvolvimento e integração",
        "ganhos_quantificados": "Redução de 15% nos gastos excessivos",
        "riscos_e_mitigacao": "Risco de resistência da equipe; mitigar com treinamento e comuni

In [6]:
# Exibir resultado
analysis_df

,problema_mapeado,oportunidade_melhoria,solucao,backlog_atividades,investimento,ganhos_quantificados,riscos_e_mitigacao
0,"Ordem de compra não é vinculada ao orçamento, ...",Implementar um sistema de vinculação automátic...,Desenvolver e integrar um módulo no sistema ER...,"[Analisar requisitos do sistema atual, Desenvo...",200 horas de desenvolvimento e integração,Redução de 15% nos gastos excessivos,Risco de resistência da equipe; mitigar com tr...
1,Falta de visibilidade em tempo real dos gastos...,Implementar dashboards em tempo real para moni...,Utilizar ferramentas de BI para criar dashboar...,"[Selecionar ferramenta de BI, Desenvolver dash...",150 horas de desenvolvimento e configuração,Melhoria de 20% na eficiência de monitoramento...,Risco de dados imprecisos; mitigar com validaç...
2,Processos de compra não padronizados levando a...,Padronizar processos de compra com base em mel...,Desenvolver e implementar um manual de procedi...,"[Revisar processos atuais, Desenvolver manual ...",100 horas para desenvolvimento e treinamento,Redução de 10% em inconsistências de compra,Risco de não adesão; mitigar com incentivos e ...
3,Comunicação ineficaz entre departamentos sobre...,Melhorar a comunicação interdepartamental com ...,Implementar uma plataforma de comunicação inte...,"[Selecionar plataforma de comunicação, Configu...",80 horas para configuração e treinamento,Aumento de 25% na eficiência de comunicação,Risco de adoção lenta; mitigar com suporte con...
4,Falta de análise de dados históricos para prev...,Implementar análise preditiva para planejament...,Utilizar ferramentas de análise preditiva para...,"[Coletar e limpar dados históricos, Desenvolve...",120 horas para desenvolvimento e integração,Melhoria de 30% na precisão do planejamento or...,Risco de modelos imprecisos; mitigar com revis...
